Use vector search option with Assistants API v2

In [1]:
import openai
import os
from openai import AzureOpenAI
import json
import pandas as pd
from dotenv import load_dotenv
import requests
import time
load_dotenv("credentials.env")

True

In [ ]:
from typing import Iterable
from openai.types import FileObject
from openai.types.beta.threads import Message, TextContentBlock, ImageFileContentBlock, Run
from openai.types.beta.thread import Thread

def format_messages(messages: Iterable[Message]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, ImageFileContentBlock):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

In [2]:
endpoint = "https://ai-njonpdaistudiohubeus2637795437656.openai.azure.com/"
client = AzureOpenAI(
  azure_endpoint=endpoint, #os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key= "2293295d97f94bd882e7390552cfc596", #os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-05-01-preview"
)



assistant = client.beta.assistants.create(
  model="gpt-4o", # replace with model deployment name.
  instructions="You are an AI assistant that retrieves information from given document and answers questions based on the information in the document.",
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_6ZDlmxqolj0gt3FYHq4hNUml"]}},
  temperature=1,
  top_p=1
)

# assistant = client.beta.assistants.update(
#   assistant_id=assistant.id,
#   tool_resources={"file_search": {"vector_store_ids": ["vs_6ZDlmxqolj0gt3FYHq4hNUml"]}},
# )


In [3]:
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="list all benefits" # Replace this with your prompt
)



# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)


SyncCursorPage[Message](data=[Message(id='msg_wsXqmJKwgHouiy8GuKOEchko', assistant_id='asst_cXxafCYiLj2cjlJsihykEYrC', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=727, file_citation=FileCitation(file_id='assistant-L48XnFjcgvv1wRjouqHLnBP5'), start_index=715, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=739, file_citation=FileCitation(file_id='assistant-L48XnFjcgvv1wRjouqHLnBP5'), start_index=727, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=1330, file_citation=FileCitation(file_id='assistant-nc8a6u6yila2h2DDf39K1ifL'), start_index=1318, text='【4:1†source】', type='file_citation'), FileCitationAnnotation(end_index=1342, file_citation=FileCitation(file_id='assistant-nc8a6u6yila2h2DDf39K1ifL'), start_index=1330, text='【4:2†source】', type='file_citation'), FileCitationAnnotation(end_index=1600, file_citation=FileCitation(file_id='assistant-nc8a6u6yila2h2DD

In [17]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
format_messages(messages)

user:
list all benefits

assistant:
The provided documents outline several benefits offered to employees:

### 1. PerksPlus Program:
The PerksPlus program is designed to support employees' health and wellness with the opportunity to expense up to $1000 for fitness-related activities. This program covers:
- Gym memberships
- Personal training sessions
- Yoga and Pilates classes
- Fitness equipment purchases
- Sports team fees
- Health retreats and spas
- Outdoor adventure activities (e.g., rock climbing, hiking, kayaking)
- Group fitness classes (e.g., dance, martial arts, cycling)
- Virtual fitness programs (e.g., online yoga and workout classes)
- Lessons and experiences like skiing, snowboarding, scuba diving, surfing, and horseback riding【4:0†source】【4:0†source】.

### 2. Health Insurance Plans:
Contoso Electronics offers two comprehensive health insurance plans via Northwind Health:

**Northwind Health Plus:**
- Comprehensive coverage for medical, vision, and dental services
- Presc